In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

2023-03-05 11:03:28.844404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 11:03:29.011892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 11:03:29.011913: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 11:03:29.047450: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-05 11:03:30.390292: W tensorflow/stream_executor/platform/de

In [ ]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
    zoom_range= 0.4,
    horizontal_flip = True,
    vertical_flip=True,
    rotation_range=60,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.4,
    fill_mode = 'nearest'
    
)


In [15]:
train= data_generator.flow_from_directory('PetImages/', class_mode= 'binary', batch_size=50, target_size=(250,250))
test= data_generator.flow_from_directory('PetImagesTest//', class_mode= 'binary', batch_size=50, target_size=(250,250))

Found 2000 images belonging to 2 classes.
Found 291 images belonging to 2 classes.


In [16]:
model = Sequential([
    Conv2D(16,(3,3), activation = 'relu', padding = 'same', input_shape=(250, 250, 3)),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(units=100, activation='relu'),
    Dense(units=1, activation='sigmoid'),

])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics =['accuracy'])


In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 250, 250, 16)      448       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 125, 125, 16)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 250000)            0         
                                                                 
 dense_2 (Dense)             (None, 100)               25000100  
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 25,000,649
Trainable params: 25,000,649
Non-trainable params: 0
__________________________________________

In [18]:
import os
from PIL import Image
folder_path = 'PetImagesTest/'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

In [ ]:
history = model.fit_generator(train, steps_per_epoch=len(train), validation_data=test, validation_steps=len(test), epochs=30)

/tmp/ipykernel_167562/3020056663.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train, steps_per_epoch=len(train), validation_data=test, validation_steps=len(test), epochs=30)


Epoch 1/30
40/40 [==============================] - 40s 902ms/step - loss: 1.2162 - accuracy: 0.5050 - val_loss: 0.7973 - val_accuracy: 0.5052
Epoch 2/30
40/40 [==============================] - 43s 1s/step - loss: 0.6939 - accuracy: 0.5795 - val_loss: 0.7313 - val_accuracy: 0.5567
Epoch 3/30
40/40 [==============================] - 44s 1s/step - loss: 0.6376 - accuracy: 0.6315 - val_loss: 0.6669 - val_accuracy: 0.5842
Epoch 4/30
40/40 [==============================] - 44s 1s/step - loss: 0.6150 - accuracy: 0.6495 - val_loss: 0.6303 - val_accuracy: 0.6460
Epoch 5/30
40/40 [==============================] - 45s 1s/step - loss: 0.5652 - accuracy: 0.7085 - val_loss: 0.6259 - val_accuracy: 0.6289
Epoch 6/30
40/40 [==============================] - 43s 1s/step - loss: 0.5338 - accuracy: 0.7385 - val_loss: 0.6660 - val_accuracy: 0.6151
Epoch 7/30
40/40 [==============================] - 43s 1s/step - loss: 0.5137 - accuracy: 0.7385 - val_loss: 0.6630 - val_accuracy: 0.6048
Epoch 8/30
40/40 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], color='green')
plt.plot(history.history['val_accuracy'], color='blue')